In [55]:
import requests, time
from bs4 import BeautifulSoup
from urllib.error import HTTPError
import pandas as pd

In [56]:
base_url = "https://www.biggerpockets.com/blog/"

In [57]:
url = f"{base_url}"
try:
    response = requests.get(url)
    response.raise_for_status()
except HTTPError as hp:
    print(hp)

In [113]:
def get_content(url):
    content = ""
    response = requests.get(url)
    page = BeautifulSoup(response.text, 'html.parser')
    for article in page.findAll('main', {'class': 'entry-content'}):
        text_elements = article.findAll(['p','ul', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'span'])
        # Extract the text from each element and concatenate them into a single string
        content = ' '.join(element.get_text() for element in text_elements)
        return content

In [114]:
page = BeautifulSoup(response.text, 'html.parser')
blog_info_ls = []
blog_info = {}
blog = page.findAll('h2', {'class': 'entry-title js-toc-ignore mt-4 mb-0 text-xl leading-snug text-left'})
cat = page.findAll('div', {'class': 'flex gap-4 justify-between items-center'})
time = page.findAll('span', {'class': 'whitespace-nowrap'})
for c, b, t in zip(cat, blog, time):
    blog_info["title"] = b.text.strip()
    blog_info["link"] = b.a['href']
    blog_info["content"] = get_content(blog_info["link"])
    blog_info["label"] = c.a.text.strip()
    blog_info["time"] = t.text.strip()
    blog_info_ls.append(blog_info.copy())

In [115]:
len(blog_info_ls)

16

In [118]:
pd.DataFrame(blog_info_ls).to_json("biggerpocket.json", orient='records', lines=True)